In [1]:
import torch
import pandas as pd

In [2]:
def import_data(file_name):
    data = pd.read_csv(file_name)
    print(data.shape)
    tensor = torch.tensor(data.values)
    tensor = tensor.float()
    return tensor
def normalize(X):
    return X/100

In [3]:
def custom_rand(a, b):
    X = torch.rand(a, b)
    return (2*X-1)
def init_params(layers_dims):
    params = dict()
    for i in range(1, len(layers_dims)):
        print(i)
        params['W' + str(i)] = custom_rand(layers_dims[i-1], layers_dims[i])
        params['b' + str(i)] = custom_rand(1, layers_dims[i])
    return params

In [4]:
def sigmoid(X):
    Z = 1/(1 + torch.exp(-X))
    return Z

In [5]:
def grads_on(params):
    for key in params:
        params[key].requires_grad = True
    return params
def update_params(params, step_size):
    for key in params:
        with torch.no_grad():
            if(params[key].requires_grad == False):
                print("ERROOOOOOOOOORRR REQUIRES GRAD SET TO 0")
            params[key] -= step_size * params[key].grad
            params[key].grad.zero_()

    return params

In [6]:
def iteration(X, Y, params, L, step_size, print_cost):
    params = grads_on(params)
    A = X
    m = X.shape[0]
    for l in range(1, L):
        Z = A@params['W' + str(l)] + params['b' + str(l)]
        A = torch.tanh(Z)
    Z = A@params['W' + str(L)] + params['b' + str(L)]
    A = sigmoid(Z)
    A = torch.clamp(A, min=1e-8)
    A = torch.clamp(A, max=1 - 1e-8)
    loss_table = -(Y*torch.log(A) + (1-Y)*torch.log(1-A))
    cost = loss_table.sum()/m
    if print_cost:
        print("THE CURRENT COST IS ", cost.item())
   # with torch.autograd.detect_anomaly():
    cost.backward()
    return update_params(params, step_size)

In [7]:
def predict(X, Y, params, L, str_type):
    A = X
    m = X.shape[0]
    for l in range(1, L):
        Z = A@params['W' + str(l)] + params['b' + str(l)]
        A = torch.tanh(Z)
    Z = A@params['W' + str(L)] + params['b' + str(L)]
    A = sigmoid(Z)
    #print("AAAAAAAAAAAA ", A)
    
    loss_table = -(Y*torch.log(A) + (1-Y)*torch.log(1-A))
    cost = loss_table.sum()/m
    Y_hat = torch.ceil(A/0.5) - 1
    incorrect = torch.abs(Y_hat - Y)
    gg = m - incorrect.sum()
    #print("THE TESTING COST IS ", cost.item())
    #print("The Model made ", gg.item(), " good predictions")
    print("(", 100*gg.item()/m, "%", str_type,"accuracy)")

In [8]:
layers_dims = [780, 60, 50, 40, 1]
params = init_params(layers_dims)
#print(params)

1
2
3
4


In [9]:
train_X = import_data("train_X.csv")
train_Y = import_data("train_Y.csv")
test_X = import_data("test_X.csv")
test_Y = import_data("test_Y.csv")

(2700, 780)
(2700, 1)
(2000, 780)
(2000, 1)


In [10]:
params = iteration(train_X, train_Y, params, len(layers_dims)-1, 0.000001, 1)

THE CURRENT COST IS  1.6185338497161865


In [13]:
for i in range(10000):
    if (i%500 == 0):
        print("EPOCH ", i)
    params = iteration(train_X, train_Y, params, len(layers_dims)-1, 0.01, (i%500 == 0))
    if (i%500 == 0):
        predict(train_X, train_Y, params, len(layers_dims)-1, "training")
        predict(test_X, test_Y, params, len(layers_dims)-1, "testing")

EPOCH  0
THE CURRENT COST IS  0.029540836811065674
( 99.96296296296296 % training accuracy)
( 59.3 % testing accuracy)
EPOCH  500
THE CURRENT COST IS  0.028237968683242798
( 99.96296296296296 % training accuracy)
( 59.4 % testing accuracy)
EPOCH  1000
THE CURRENT COST IS  0.02702975831925869
( 99.96296296296296 % training accuracy)
( 59.4 % testing accuracy)
EPOCH  1500
THE CURRENT COST IS  0.025906739756464958
( 99.96296296296296 % training accuracy)
( 59.35 % testing accuracy)


KeyboardInterrupt: 